In [ ]:
import cmocean as cmo #v2.0

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
import time as tm
# import gsw as sw
from scipy.optimize import curve_fit
from scipy.stats import chi2

import numpy.ma as ma
from geopy import distance
import mpl_toolkits
import geopy

from mpl_toolkits.basemap import Basemap

from netCDF4 import Dataset 

from matplotlib import rcParams, rc
#rcParams.update({'figure.autolayout': True})
#rcParams.update({'font.family': 'sans-serif'})
#rcParams.update({'font.sans-serif': 'TArisl'})
#rcParams.update({'text.color':'w'})
#rc('text', usetex=True)

%matplotlib inline
#from parcels import FieldSet, ParticleSet, Variable, JITParticle, AdvectionRK4, plotTrajectoriesFile
import numpy as np
import math
from datetime import timedelta
from operator import attrgetter

In [ ]:
# import stations coordinate data

col_names = pd.read_csv('./t2t9_latlong.csv', nrows=0).columns
types_dict = {'lat': float, 'lon': float,'loc': int}
types_dict.update({col: str for col in col_names if col not in types_dict})
coords = pd.read_csv('./t2t9_latlong.csv', dtype=types_dict)


coords

# load the satellite NPP data

#fname = './EccoData/global-reanalysis-phy-001-030-daily_1630280369905.nc'
fname = './EccoData/global-reanalysis-phy-001-030-daily_1629769022292.nc'
cnms_montly = Dataset(fname, mode='r')

# chl = chl_data.variables['chlor_a']
# lats = chl_data.variables['lat'][:]
# lons = chl_data.variables['lon'][:]

# lon, lat = np.meshgrid(lons, lats)
# #Changed chl to chl[:][:], wouldn't except < between the data types
# bad = (chl[:][:]<0)
# chla = np.ma.masked_where(bad, chl)
#cnms_montly.variables['time'].units

temp = cnms_montly.variables['thetao']
lats = cnms_montly.variables['latitude'][:]
lons = cnms_montly.variables['longitude'][:]
ssh = cnms_montly.variables['zos']
sal = cnms_montly.variables['so']
lon, lat = np.meshgrid(lons, lats)
np.min(temp[6,0,:,:])
print(cnms_montly.variables['time'][:])
print(np.min(lats))


time_convert
#12 is T2
#25 is 7/20 or T9

In [ ]:
fig2,ax = plt.subplots(1, figsize=(20, 15), dpi = 600)

m = Basemap(projection='merc',llcrnrlat=37,urcrnrlat=39.5,\
            llcrnrlon=-127,urcrnrlon=-120.75,lat_ts=0,resolution='h')
m.drawcoastlines()
m.drawmapboundary(fill_color='white')
m.fillcontinents(color='grey',lake_color='white')

xm, ym = m(coords['lon'].values, coords['lat'].values)
m.plot(xm,ym,'ok', markersize = 9, mfc='none', mew = 2)

n = coords['Station']

numlog = 1
for i, txt in enumerate(n):
    if (numlog % 2) == 0:
        #do nothing
        numlog = numlog+1
    else:
        if coords['loc'][i]==1:
            ax.annotate(txt, (xm[i]-20000,ym[i]+6000), fontsize = 15, fontweight = 'bold', color = 'k')
        if coords['loc'][i]==2:
            ax.annotate(txt, (xm[i]-20000,ym[i]+15000), fontsize = 15, fontweight = 'bold', color = 'k')
        if coords['loc'][i]==0:
            ax.annotate(txt, (xm[i]-20000,ym[i]+4000), fontsize = 15, fontweight = 'bold', color = 'k')
        if numlog == 23:
            numlog = 1
        else:
            numlog = numlog+1

xc, yc = m(lon, lat)
CS1 = m.pcolormesh(xc, yc, temp[6,0,:,:], shading='gouraud',cmap = cmo.cm.thermal,vmin = 8,vmax=18, zorder = 1, alpha = 0.80)

cs = m.contour(xc,yc,ssh[6,:,:],15,linewidths=1,colors = 'Black',levels=[-0.1,-0.075,-0.05,-0.025,0,0.025,0.05,0.075,0.1],linestyles = ['dashed','dashed','dashed','dashed','solid','solid','solid','solid','solid'],zorder = 2)
ax.clabel(cs, cs.levels, inline=True, fontsize=16)

# cs = m.contour(xc,yc,ssh[6,:,:],15,linewidths=1,colors = 'Black',zorder = 2)
# ax.clabel(cs, cs.levels, inline=True, fontsize=10)

# xii, yii = m(lon_mdt, lat_mdt)
# # mm = m.contourf(xii,yii,mdt, cmap = mcon, vmin = -0.5, vmax = 1.75, lw = 3, zorder = 0, alpha = 0.5)
# # m.colorbar(mm)

# m.contour(xii,yii,mdt,levels = np.arange(-1,2, 0.025), colors = 'grey', linewidths = 3, zorder = 2, alpha = 1)

m.drawparallels(np.arange(-50,80,1),labels=[1,0,0,0], fontsize=24)
m.drawmeridians(np.arange(-180,180,2),labels=[0,0,0,1], fontsize=24)

cbar = m.colorbar(CS1, ticks = [10,15,20,25,30],label = 'Temperature (C)')
cbar.set_label(r'Temperature (C)', fontsize = 24)
cbar.ax.tick_params(labelsize = 24)

plt.tick_params(axis='both', which='major', labelsize=24)
plt.tick_params(axis='both', which='minor', labelsize=24)
plt.xlabel('Longitude', fontsize = 24, labelpad = 40)
plt.ylabel('Latitude', fontsize = 24, labelpad = 70)

fig2.savefig('./cruise_map_t2_temperature_7_7.jpg', format='jpeg', dpi = 600)

In [ ]:
fig2,ax = plt.subplots(1, figsize=(20, 15), dpi = 600)

m = Basemap(projection='merc',llcrnrlat=37,urcrnrlat=39.5,\
            llcrnrlon=-127,urcrnrlon=-120.75,lat_ts=0,resolution='h')
m.drawcoastlines()
m.drawmapboundary(fill_color='white')
m.fillcontinents(color='grey',lake_color='white')

xm, ym = m(coords['lon'].values, coords['lat'].values)
m.plot(xm,ym,'ok', markersize = 9, mfc='none', mew = 2)

n = coords['Station']

numlog = 1
for i, txt in enumerate(n):
    if (numlog % 2) == 0:
        #do nothing
        numlog = numlog+1
    else:
        if coords['loc'][i]==1:
            ax.annotate(txt, (xm[i]-20000,ym[i]+6000), fontsize = 15, fontweight = 'bold', color = 'k')
        if coords['loc'][i]==2:
            ax.annotate(txt, (xm[i]-20000,ym[i]+15000), fontsize = 15, fontweight = 'bold', color = 'k')
        if coords['loc'][i]==0:
            ax.annotate(txt, (xm[i]-20000,ym[i]+4000), fontsize = 15, fontweight = 'bold', color = 'k')
        if numlog == 23:
            numlog = 1
        else:
            numlog = numlog+1

xc, yc = m(lon, lat)
CS1 = m.pcolormesh(xc, yc, sal[6,0,:,:], shading='gouraud',cmap = cmo.cm.haline,vmin = 32,vmax=34, zorder = 1, alpha = 0.8)

cs = m.contour(xc,yc,ssh[6,:,:],15,linewidths=1,colors = 'Black',levels=[-0.1,-0.075,-0.05,-0.025,0,0.025,0.05,0.075,0.1],linestyles = ['dashed','dashed','dashed','dashed','solid','solid','solid','solid','solid'],zorder = 2)
ax.clabel(cs, cs.levels, inline=True, fontsize=16)

# cs = m.contour(xc,yc,ssh[6,:,:],15,linewidths=1,colors = 'Black',zorder = 2)
# ax.clabel(cs, cs.levels, inline=True, fontsize=10)

# xii, yii = m(lon_mdt, lat_mdt)
# # mm = m.contourf(xii,yii,mdt, cmap = mcon, vmin = -0.5, vmax = 1.75, lw = 3, zorder = 0, alpha = 0.5)
# # m.colorbar(mm)

# m.contour(xii,yii,mdt,levels = np.arange(-1,2, 0.025), colors = 'grey', linewidths = 3, zorder = 2, alpha = 1)

m.drawparallels(np.arange(-50,80,1),labels=[1,0,0,0], fontsize=24)
m.drawmeridians(np.arange(-180,180,2),labels=[0,0,0,1], fontsize=24)

cbar = m.colorbar(CS1,label = 'Salinity (PSU)')
cbar.set_label(r'Salinity (PSU)', fontsize = 24)
cbar.ax.tick_params(labelsize = 24)

plt.tick_params(axis='both', which='major', labelsize=24)
plt.tick_params(axis='both', which='minor', labelsize=24)
plt.xlabel('Longitude', fontsize = 24, labelpad = 40)
plt.ylabel('Latitude', fontsize = 24, labelpad = 70)

fig2.savefig('./cruise_map_t2_sal_7_7.jpg', format='jpeg', dpi = 600)